In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import argparse
from tqdm import tqdm
import pandas as pd

import torch
import torch.optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SequentialSampler
from torch.nn import DataParallel
import torch.nn.functional as F
from torch.autograd import Variable

from config.config import *
from utils.common_util import *
from networks.imageclsnet import init_network
from datasets.protein_dataset import ProteinDataset
from utils.augment_util import *
from utils.log_util import Logger

run on 2fb1688be1a5


In [3]:
# All label names in the public HPA and their corresponding index.
all_locations = dict({
    "Nucleoplasm": 0,
    "Nuclear membrane": 1,
    "Nucleoli": 2,
    "Nucleoli fibrillar center": 3,
    "Nuclear speckles": 4,
    "Nuclear bodies": 5,
    "Endoplasmic reticulum": 6,
    "Golgi apparatus": 7,
    "Intermediate filaments": 8,
    "Actin filaments": 9,
    "Focal adhesion sites": 9,
    "Microtubules": 10,
    "Mitotic spindle": 11,
    "Centrosome": 12,
    "Centriolar satellite": 12,
    "Plasma membrane": 13,
    "Cell Junctions": 13,
    "Mitochondria": 14,
    "Aggresome": 15,
    "Cytosol": 16,
    "Vesicles": 17,
    "Peroxisomes": 17,
    "Endosomes": 17,
    "Lysosomes": 17,
    "Lipid droplets": 17,
    "Cytoplasmic bodies": 17,
    "No staining": 18,
    # markpeng
    "Rods & rings": 18,
})

In [4]:
datasets_names = ['test', 'val']
split_names = ['random_ext_folds5', 'random_ext_noleak_clean_folds5']
augment_list = ['default', 'flipud', 'fliplr','transpose', 'flipud_lr',
                'flipud_transpose', 'fliplr_transpose', 'flipud_lr_transpose']

parser = argparse.ArgumentParser(description='PyTorch Protein Classification')
parser.add_argument('--out_dir', type=str, help='destination where predicted result should be saved')
parser.add_argument('--gpu_id', default='0', type=str, help='gpu id used for predicting (default: 0)')
parser.add_argument('--arch', default='class_densenet121_dropout', type=str,
                    help='model architecture (default: class_densenet121_dropout)')
parser.add_argument('--num_classes', default=28, type=int, help='number of classes (default: 28)')
parser.add_argument('--in_channels', default=4, type=int, help='in channels (default: 4)')
parser.add_argument('--img_size', default=768, type=int, help='image size (default: 768)')
parser.add_argument('--crop_size', default=512, type=int, help='crop size (default: 512)')
parser.add_argument('--batch_size', default=32, type=int, help='train mini-batch size (default: 32)')
parser.add_argument('--workers', default=3, type=int, help='number of data loading workers (default: 3)')
parser.add_argument('--fold', default=0, type=int, help='index of fold (default: 0)')
parser.add_argument('--augment', default='default', type=str, help='test augmentation (default: default)')
parser.add_argument('--seed', default=100, type=int, help='random seed (default: 100)')
parser.add_argument('--seeds', default=None, type=str, help='predict seed')
parser.add_argument('--dataset', default='test', type=str, choices=datasets_names,
                    help='dataset options: ' + ' | '.join(datasets_names) + ' (default: test)')
parser.add_argument('--split_name', default='random_ext_folds5', type=str, choices=split_names,
                    help='split name options: ' + ' | '.join(split_names) + ' (default: random_ext_folds5)')
parser.add_argument('--predict_epoch', default=None, type=int, help='number epoch to predict')

_StoreAction(option_strings=['--predict_epoch'], dest='predict_epoch', nargs=None, const=None, default=None, type=<class 'int'>, choices=None, help='number epoch to predict', metavar=None)

In [5]:
def predict(test_loader, model, submit_out_dir, dataset):
    all_probs = []
    img_ids = np.array(test_loader.dataset.img_ids)
    for it, iter_data in tqdm(enumerate(test_loader, 0),
                              total=len(test_loader)):
        images, indices = iter_data
        images = Variable(images.cuda(), volatile=True)
        outputs = model(images)
        logits = outputs

        probs = F.sigmoid(logits).data
        all_probs += probs.cpu().numpy().tolist()
    img_ids = img_ids[:len(all_probs)]
    all_probs = np.array(all_probs).reshape(len(img_ids), -1)

    np.save(opj(submit_out_dir, 'prob_%s.npy' % dataset), all_probs)

    result_df = prob_to_result(all_probs, img_ids)
    result_df.to_csv(opj(submit_out_dir, 'results_%s.csv.gz' % dataset),
                     index=False,
                     compression='gzip')


def prob_to_result(probs, img_ids, th=0.5):
    probs = probs.copy()
    probs[np.arange(len(probs)), np.argmax(probs, axis=1)] = 1

    pred_list = []
    for line in probs:
        s = ' '.join(list([str(i) for i in np.nonzero(line > th)[0]]))
        pred_list.append(s)
    result_df = pd.DataFrame({ID: img_ids, PREDICTED: pred_list})
    return result_df

In [6]:
test_dataset_path = "/workspace/Kaggle/HPA/hpa_2020/test"
args = parser.parse_args([
    "--out_dir", "external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds",
    "--gpu_id", "0",
    "--arch", "class_densenet121_dropout",
    "--img_size", "768",
    "--crop_size", "512",
    "--seeds", "1120",
    "--batch_size", "8",
    "--fold", "0",
    "--dataset", "test"
])
args

Namespace(arch='class_densenet121_dropout', augment='default', batch_size=8, crop_size=512, dataset='test', fold=0, gpu_id='0', img_size=768, in_channels=4, num_classes=28, out_dir='external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds', predict_epoch=None, seed=100, seeds='1120', split_name='random_ext_folds5', workers=3)

In [7]:
# args.predict_epoch = 'final' if args.predict_epoch is None else '%03d' % args.predict_epoch
# network_path = opj(RESULT_DIR, 'models', args.out_dir, 'fold%d' % args.fold,
#                    '%s.pth' % args.predict_epoch)
network_path = "/workspace/Kaggle/HPA/bestfitting/models/external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/fold0/final.pth"

# setting up the visible GPU
os.environ['CUDA_VISIBLE_DEVICES'] = args.gpu_id

model_params = {}
model_params['architecture'] = args.arch
model_params['num_classes'] = args.num_classes
model_params['in_channels'] = args.in_channels
model = init_network(model_params)

# log.write(">> Loading network:\n>>>> '{}'\n".format(network_path))
checkpoint = torch.load(network_path)
model.load_state_dict(checkpoint['state_dict'])
# log.write(">>>> loaded network:\n>>>> epoch {}\n".format(checkpoint['epoch']))

# moving network to gpu and eval mode
# model = DataParallel(model)
model.cuda()
model.eval()

>> Using pre-trained model.


DensenetClass(
  (backbone): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(4, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momen

In [8]:
# Data loading code
dataset = args.dataset
# if dataset == 'test':
#     test_split_file = opj(DATA_DIR, 'split', 'test_11702.csv')
# elif dataset == 'val':
#     test_split_file = opj(DATA_DIR, 'split', args.split_name, 'random_valid_cv%d.csv' % args.fold)
# else:
#     raise ValueError('Unsupported or unknown dataset: {}!'.format(dataset))
test_split_file = "/workspace/Kaggle/HPA/hpa_2020/sample_submission.csv"


test_dataset = ProteinDataset(
    test_split_file,
    img_size=args.img_size,
    is_trainset=(dataset != 'test'),
    return_label=False,
    in_channels=args.in_channels,
    transform=None,
    crop_size=args.crop_size,
    random_crop=False,
)
test_loader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=args.batch_size,
    drop_last=False,
    num_workers=args.workers,
    pin_memory=True,
)

/workspace/Kaggle/HPA/hpa_2020/inference/test/images_768
/workspace/Kaggle/HPA/hpa_2020/train/external_v18_768


In [ ]:
seeds = [args.seed] if args.seeds is None else [int(i) for i in args.seeds.split(',')]
for seed in seeds:
    test_dataset.random_crop = (seed != 0)
    for augment in args.augment:
        test_loader.dataset.transform = eval('augment_%s' % augment)
        if args.crop_size > 0:
            sub_submit_out_dir = opj(submit_out_dir, '%s_seed%d' % (augment, seed))
        else:
            sub_submit_out_dir = opj(submit_out_dir, augment)
        if not ope(sub_submit_out_dir):
            os.makedirs(sub_submit_out_dir)
        with torch.no_grad():
            predict(test_loader, model, sub_submit_out_dir, dataset)

In [ ]:
!ls /workspace/Kaggle/HPA/hpa_2020/test/inference/images_768